# Using `astropy.coordinates` to Query FUSE Observations
This notebook walks through selecting stars from the master list of FUSE observations, obtained from MAST. 

In [2]:
##Set up matplotlib for display in the browser:
%matplotlib inline

Standard modules:

In [3]:
##Very most basic/standard imports:
import matplotlib.pyplot as plt
import numpy as np

##If you read FITS or ascii tables, these are necessary:
from astropy.io import fits
from astropy.io import ascii

##Automatic unit tracking...maybe?
import astropy.units as u

##Module containing a lot of standard stats packages:
from scipy import stats

#System options:
import sys

Specifically for constructing coordinate queries: import functionality from the `astropy.coordinates` module.

In [4]:
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.coordinates as coord

## FUSE exposure catalog search for Perseus targets

Here I'm searching the FUSE exposure catalog (downloaded through MAST) to search for targets at low-latitude in the Perseus arm. 

I'm particularly intersted in a region within `165 < l < 175` or so and at low latitudes. I'll do this by using `astropy.coordinates` modules to transform the RA/Dec of the targets into Galactic longitude and latitude, then query on that.

### Step 1: use  `astropy.ascii.read` to read in the exposure catalog.
This turns out to be really easy, since MAST spits this out as a CSV file. We will read this in with `ascii.read`, then test the properties of the resulting table (be prepared for a pop-up of sorts below).

In [17]:
fuse=ascii.read('FUSE_exposureCatalog.csv')
fuse?

Let's look at the column names, then print a sample entry.

In [6]:
fuse.colnames

['dataproduct_type',
 'obs_collection',
 'instrument_name',
 'project',
 'filters',
 'wavelength_region',
 'target_name',
 'target_classification',
 'obs_id',
 's_ra',
 's_dec',
 'proposal_id',
 'proposal_pi',
 'calib_level',
 't_min',
 't_max',
 't_exptime',
 'em_min',
 'em_max',
 't_obs_release',
 's_region',
 'jpegURL',
 'obsid',
 'objID']

In [7]:
fuse[5]

dataproduct_type,obs_collection,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,proposal_id,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,t_obs_release,s_region,jpegURL,obsid,objID
str8,str4,str3,int64,int64,str2,str23,str34,str11,float64,float64,str4,str15,int64,float64,float64,int64,float64,float64,float64,str65,str81,int64,int64
spectrum,FUSE,FUV,--,--,UV,0513-69,X-RAY SOURCE,b0160102000,78.461667,-69.863056,B016,Hutchings,2,52213.1093403,52213.2625133,3965,90.0,119.0,nan,CIRCLE ICRS 78.46166700000001 -69.863056 0.00416666666667,http://archive.stsci.edu/browse/previews/fuse/b0160102/b016010200000specttagf.gif,3001000470,3001000337


### Step 2: Extract RA/Dec list into the `astropy.coordinates` system

Here we use the `SkyCoord` function to incorporate our positions into the `astropy.coordinates` system. This is useful for facilitating coordinate conversions and manipulations.

In [23]:
fuseCoords=SkyCoord(fuse['s_ra'],fuse['s_dec'],unit=(u.deg,u.deg),frame='fk5')

Take a look at the structure of the output for one of the pointing positions:

In [24]:
fuseCoords[5],fuseCoords[5].galactic

(<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
     (78.461667, -69.863056)>, <SkyCoord (Galactic): (l, b) in deg
     (280.79643055, -33.68811397)>)

Here's an example of extracting those coordinates for individual use.

In [26]:
fuseCoords.galactic[5],fuseCoords.galactic.l[5]

(<SkyCoord (Galactic): (l, b) in deg
     (280.79643055, -33.68811397)>, <Longitude 280.7964305504501 deg>)

### Step 3: add the (l,b) values into the table
This shows us how we can readily convert the RA/Dec values to galactic coordinates. Now we'll put the Galactic coordinates back into the `fuse` table.  Be aware: this does not (evidently) save the units!

In [18]:
fuse['gal_longitude'] = fuseCoords.galactic.l
fuse['gal_latitude'] = fuseCoords.galactic.b
fuse.colnames

['dataproduct_type',
 'obs_collection',
 'instrument_name',
 'project',
 'filters',
 'wavelength_region',
 'target_name',
 'target_classification',
 'obs_id',
 's_ra',
 's_dec',
 'proposal_id',
 'proposal_pi',
 'calib_level',
 't_min',
 't_max',
 't_exptime',
 'em_min',
 'em_max',
 't_obs_release',
 's_region',
 'jpegURL',
 'obsid',
 'objID',
 'gal_longitude',
 'gal_latitude']

### Step 4: select objects in the longitude/latitude range of interest (165 < l < 175, -15 < b < 15)
There are two approaches to this. First we can use the `fuseCoords` coordinate variable to apply the constraints. 

In [28]:
perseusB=((np.absolute(fuseCoords.galactic.l-170.*u.deg) <= 5.*u.deg) &  
          (np.absolute(fuseCoords.galactic.b) <= 15.*u.deg))
perseusB.sum() ## This let's us tell how many meet the criteria.

31

Second we can use the columns in the `fuse` table to apply the constraints. (N.B. We do not use units on our l,b in this case!)

In [27]:
perseusB=((np.absolute(fuse['gal_longitude']-170.) <= 5.) &  
          (np.absolute(fuse['gal_latitude']) <= 15.))
perseusB.sum() ## This let's us tell how many meet the criteria.

31

### Step 5: place constraints on the nature of the objects
So, there are 31 objects observed in that coordinate range. Let's exclude some categories of object that aren't useful to further constrain things. 

*There is almost certainly a better way to select against these classifications. Perhaps creating a list of the bad ones and testing against that??*

In [15]:
perseusB=((np.absolute(fuse['gal_longitude']-170.) <= 10.) &  
          (np.absolute(fuse['gal_latitude']) <= 15.) & 
          (fuse['target_classification'] != 'T TAURI STARS') &
          (fuse['target_classification'] != 'H II REGION') &
          (fuse['target_classification'] != 'H II REGION') &
          (fuse['target_classification'] != 'WDA') &
          (fuse['target_classification'] != 'WDB') &
          (fuse['target_classification'] != 'G I-III') &
          (fuse['target_classification'] != 'F0-2') &
          (fuse['target_classification'] != 'A0-3 IV-V') &
          (fuse['target_classification'] != 'CENT. STAR PLAN. NEB.') &          
          (fuse['target_classification'] != 'SKY BACKGROUND') &
          (fuse['target_classification'] != 'DWARF NOVAE') &
          (fuse['target_classification'] != 'B0-B2 V-IV') &
          (fuse['target_classification'] != 'AE'))

perseusB.sum() ## This let's us tell how many meet the criteria.

17

**Printing the selected objects:**
* Print only those objects in `perseusB`
* Print a subset of the columns
* Don't show units
* Print *all* rows, no matter how many.

In [29]:
fuse[perseusB]['target_name','target_classification',
               'gal_longitude','gal_latitude'].pprint(show_unit=False,
                                                      max_lines=-1)



target_name target_classification gal_longitude  gal_latitude 
----------- --------------------- ------------- --------------
     GM-AUR         T TAURI STARS 172.567500932 -8.19333015368
     GM-AUR         T TAURI STARS 172.567500932 -8.19333015368
   HD034656          SUPERGIANT O 170.037986535  0.27053335684
   HD039659 CENT. STAR PLAN. NEB. 166.156263836  10.4753337454
   HD039659 CENT. STAR PLAN. NEB. 166.156263836  10.4753337454
   HD039659 CENT. STAR PLAN. NEB. 166.156263836  10.4753337454
    HD27786                   WDA 165.530700745 -10.7471520766
    HD30675           B3-B5 III-I 173.605617482  -10.205209094
    HD31293                    AE 172.499043809 -7.98079083593
    HD31293                    AE 172.499044162 -7.98079111031
    HD31647             A0-3 IV-V 167.150501594 -2.87683883077
    HD33357  INTERACTING BINARIES 165.595575235  2.17376914755
    HD33357  INTERACTING BINARIES 165.595575235  2.17376914755
   HD33959C                   WDA 173.298827491 -3.3574

#### NOTES:
1. HD34078 is AE Aur. Not a good candidate? 
2. HD34656 seems to be too close, not behind Perseus [d~2.3 kpc from Bowen+ (2008), Patriarchi+ (2003)].
3. HD 41161 is too close at d~1.6 kpc.
4. HD 33853 is an Algol binary...ugly.
